# Ex02

## create a connection to the database using the library sqlite3

In [1]:
import sqlite3
import pandas as pd

connection = sqlite3.connect('../data/logs.db')
cursor = connection.cursor()

cursor.execute("ATTACH DATABASE '../data/checking-logs.sqlite' AS logs")

connection.commit()

## create a new table datamart in the database by joining the tables pageviews and checker using only one query

In [67]:
query_drop = "DROP TABLE IF EXISTS datamart;"
cursor.execute(query_drop)

query = """
    CREATE TABLE datamart AS
    SELECT 
        c.uid,
        c.labname, 
        MIN(c.timestamp) AS first_commit_ts, 
        MIN(p.datetime) AS first_view_ts  
    FROM checker AS c
    FULL OUTER JOIN pageviews AS P
    ON c.uid = p.uid
    WHERE
        c.status = 'ready'
        AND c.uid LIKE 'user_%' 
        AND c.numTrials = 1
        AND c.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
    GROUP BY 
        c.uid, c.labname;
"""
cursor.execute(query)

datamart = pd.io.sql.read_sql("SELECT * FROM datamart", connection)
datamart['first_commit_ts'] = pd.to_datetime(df['first_commit_ts'])
datamart['first_view_ts'] = pd.to_datetime(df['first_view_ts'])

print(datamart)

connection.commit() 

        uid   labname            first_commit_ts              first_view_ts
0    user_1    laba04 2020-04-26 17:06:18.462708 2020-04-26 21:53:59.624136
1    user_1   laba04s 2020-04-26 17:12:11.843671 2020-04-26 21:53:59.624136
2    user_1    laba05 2020-05-02 19:15:18.540185 2020-04-26 21:53:59.624136
3    user_1    laba06 2020-05-17 16:26:35.268534 2020-04-26 21:53:59.624136
4    user_1   laba06s 2020-05-20 12:23:37.289724 2020-04-26 21:53:59.624136
..      ...       ...                        ...                        ...
135  user_8   laba04s 2020-04-19 10:22:35.761944                        NaT
136  user_8    laba05 2020-05-02 13:28:07.705193                        NaT
137  user_8    laba06 2020-05-16 17:56:15.755553                        NaT
138  user_8   laba06s 2020-05-16 20:01:07.900727                        NaT
139  user_8  project1 2020-05-14 15:42:04.002981                        NaT

[140 rows x 4 columns]


## using Pandas methods, create two dataframes: test and control


In [74]:
test = datamart.dropna(subset=['first_view_ts'])
control = datamart[datamart['first_view_ts'].isna()]

average_first_view_ts = test['first_view_ts'].mean()
control.loc[control['first_view_ts'].isna(), 'first_view_ts'] = average_first_view_ts

test.to_sql('test',con=connection, if_exists='replace', index=False)
control.to_sql('control',con=connection, if_exists='replace', index=False)

connection.commit()

## close the connection


In [75]:
connection.close()